# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [221]:
# import libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sqlalchemy import create_engine

from gensim.test.utils import datapath
from gensim import utils
import gensim.models

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import joblib

[nltk_data] Downloading package punkt to /home/tarciso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tarciso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarciso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='MessageCategoryWide')
X = messages_df.message.values
Y = categories_df.drop('message_id', axis=1).values
category_columns = categories_df.columns

In [3]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [131]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 4. Split dataset
- Split data into train and test sets
- Train pipeline

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=199)

In [133]:
X_train

array(['crudess you are the most charm and charismatic DT member .. thanks for all your videos in Santiago.. and to talk with me twice',
       'Tidal waves triggered by an earthquake off Indonesia on Sunday swept over a vast swath of coastlines, including those in India, Indonesia, Malaysia, the Maldives, Somalia, Sri Lanka and Thailand.',
       'The earthquake happened at 4h34 in the evening and for the night. ',
       ...,
       'Earthquake of the day #Haiti or Google possibly leaving China ?',
       'When the matter had been debated previously, the Non-Aligned Movement foreign ministers firmly stated that there was no right to humanitarian intervention.',
       '@DumboNYC : Oh no RT @endtwist : ConEd power station in #DUMBO is flooding ! #nopower #sandy'],
      dtype=object)

In [134]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [135]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 36)


### 2. Write a tokenization function to process your text data

In [136]:
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [137]:
def tokenize_str(text):
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [138]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [139]:
tokenize_str(X[0])

'weather update cold front cuba could pas haiti'

### 3. Generate tokenized messages table for further analysis

In [9]:
messages_df['tokens'] = messages_df.message.apply(lambda x: tokenize_str(x))

In [10]:
messages_df

,message_id,message,original,genre,num_words,tokens
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13,weather update cold front cuba could pas haiti
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9,hurricane
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6,looking someone name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13,un report leogane 80 90 destroyed hospital st ...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12,say west side haiti rest country today tonight
...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21,training demonstrated enhance micronutrient pr...
26140,30262,A suitable candidate has been selected and OCH...,None,news,22,suitable candidate selected ocha jakarta curre...
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23,proshika operating cox bazar municipality 5 un...
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31,2 000 woman protesting conduct election tearga...


In [11]:
messages_tokens = messages_df[['message_id','tokens']]

In [12]:
messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

### 3. Find n-grams and save to DB

In [13]:
def get_ngrams_freqs(messages_array, n=1):
    vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
    bag_of_words = vec.transform(messages_array)
    word_count = bag_of_words.sum(axis=0)
    words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
    words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
    return words_freq_df

In [14]:
unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens)
bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens, n=2)
trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens, n=3)
ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [15]:
ngrams_freqs

,ngram,n,count
0,water,1,3034
1,people,1,2998
2,food,1,2892
3,help,1,2649
4,need,1,2484
...,...,...,...
289130,crisis identifying crisis,3,1
289131,identifying crisis function,3,1
289132,crisis function hiv,3,1
289133,function hiv aid,3,1


In [16]:
ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 3. Train a Word2Vec model using messages text

In [140]:
class Message(object):
    """An interator that yields messages tokens (lists of str)."""
    
    def __init__(self, messages_df, sample_size=-1):
        self.messages_df = messages_df
        self.sample_size = sample_size

    def __iter__(self):
        messages = messages_df
        
        #Sample dataset if specified
        if self.sample_size > 0:
            messages = messages.sample(self.sample_size)
        
        #Yeld tokenized message joined by whitespaces
        for token_str in messages.tokens:
            yield token_str

In [141]:
sentences = Message2(messages_df=messages_df)
w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
w2v_model_full.save("messages-word2vec-full.model")

In [142]:
w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [144]:
#?w2v_model_full

In [145]:
for i, word in enumerate(w2v_model_full.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [146]:
w2v_model_full.wv['earthquake']

array([ 0.5672106 ,  0.29789364,  0.463643  ,  0.19121738, -0.58913875,
        0.86039734,  0.40682167, -0.57124066,  1.2928406 ,  0.45593792,
        1.0122268 , -0.3228534 , -1.087786  , -0.35704237,  1.0200481 ,
       -1.1935841 , -0.02619644, -0.20060593, -0.10999487,  0.62313074,
        0.03346012,  0.59004444,  0.19915423, -0.5932419 ,  0.75793874,
       -0.26756352,  0.36034623,  0.32203496, -0.27114403,  0.55780137,
       -1.3636817 , -0.35188434,  0.33116597, -0.20347492, -0.14132884,
       -0.4912949 , -0.05714604,  0.15034439, -0.01098222,  0.47501513,
       -0.35004178,  0.39268813, -0.04542985,  0.761094  , -0.50398266,
       -0.6441565 ,  0.64477974, -0.51708215, -0.18178183,  0.3622531 ,
       -0.52592355, -1.5337172 ,  0.7676356 ,  0.15075684, -0.231194  ,
       -0.16355547, -0.78194165,  0.8512226 , -0.9633786 ,  0.41826957,
       -0.9286241 ,  0.4154575 ,  0.22709647,  0.41657946,  0.01742937,
        0.2738443 , -1.082624  , -0.57317954,  0.81713694, -0.47

##### Implement Vector Aggregator

In [170]:
np.array([1,2])[0]

1

In [168]:
len(X)

26144

In [172]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [174]:
X.shape[0]

26144

In [226]:
from collections import defaultdict

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec_model, num_dims):
        self.word2vec_model = word2vec_model
        self.num_dims = num_dims
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model.wv[w] for w in doc_tokens if w in self.word2vec_model.wv]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
    
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec_model, num_dims):
        self.word2vec_model = word2vec_model
        self.num_dims = num_dims
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model.wv[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model.wv]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [227]:
test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
test_dict['Hi']

1

##### Use a Pipeline to train the Word2Vec estimator

In [228]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

w2v_mean_pip = Pipeline([
    ('w2v_mean', TfidfEmbeddingVectorizer(w2v_model_full, w2v_model_full.wv.vectors.shape[1])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

In [229]:
%%time

w2v_mean_pip.fit(X_train, y_train)

CPU times: user 12.6 s, sys: 7.14 ms, total: 12.6 s
Wall time: 12.6 s


Pipeline(memory=None,
         steps=[('w2v_mean',
                 <__main__.TfidfEmbeddingVectorizer object at 0x7f329d07b450>),
                ('clf',
                 MultiOutputClassifier(estimator=GaussianNB(priors=None,
                                                            var_smoothing=1e-09),
                                       n_jobs=None))],
         verbose=False)

In [230]:
%%time

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = w2v_mean_pip.predict(X_test)

#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=category_columns))
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

              precision    recall  f1-score   support

message_id-0       0.28      0.50      0.36      1532
message_id-1       0.80      0.55      0.65      4950

   micro avg       0.57      0.54      0.55      6482
   macro avg       0.54      0.53      0.51      6482
weighted avg       0.67      0.54      0.58      6482

              precision    recall  f1-score   support

   related-0       0.83      0.94      0.88      5393
   related-1       0.22      0.08      0.12      1143

    accuracy                           0.79      6536
   macro avg       0.52      0.51      0.50      6536
weighted avg       0.72      0.79      0.75      6536

              precision    recall  f1-score   support

   request-0       1.00      0.14      0.25      6506
   request-1       0.01      1.00      0.01        30

    accuracy                           0.15      6536
   macro avg       0.50      0.57      0.13      6536
weighted avg       1.00      0.15      0.25      6536

              preci

/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

infrastructure_related-0       0.97      0.10      0.18      6219
infrastructure_related-1       0.05      0.94      0.10       317

                accuracy                           0.14      6536
               macro avg       0.51      0.52      0.14      6536
            weighted avg       0.93      0.14      0.18      6536

              precision    recall  f1-score   support

 transport-0       0.95      0.09      0.17      6209
 transport-1       0.05      0.91      0.10       327

    accuracy                           0.13      6536
   macro avg       0.50      0.50      0.13      6536
weighted avg       0.91      0.13      0.17      6536

              precision    recall  f1-score   support

 buildings-0       0.97      0.11      0.20      6414
 buildings-1       0.02      0.84      0.03       122

    accuracy                           0.13      6536
   macro avg       0.50      0.48      0.12      6536
we

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
X_train.shape

(19662,)

In [14]:
X_train

array(['It works as a fertilizer enhancement, a composting additive, and even as an alternative to household cleaning products, according to EMRO.',
       'I need a job to feed my family. I am an electrician and a musician. I perform harmonization and orchestration in Mexico. I speak English and French. Let me know how I can help',
       'The report also warns that if the clan fighting extends to sorghum-producing areas in the south during this critical stage of crop harvest, it could further hamper harvesting and exert a negative impact on food security throughout Somalia.',
       ...,
       'HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time',
       'The low snowfall pattern and the snowfall differentiation among catchments are clearly reflected in the irrigated wheat production levels.',
       "Having fun at #Lexi 's ! Real food in real house with electrical lighting . Good times . I feel human

In [15]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

ValueError: WRITEBACKIFCOPY base is read-only

In [ ]:
print(X_test.shape)

In [ ]:
print(y_pred.shape)
y_pred

In [ ]:
y_pred[:,0]

In [ ]:
y_test

In [ ]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'multi_clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'multi_clf__estimator__bootstrap', 'multi_clf__estimator__class_weight', 'multi_clf__estimator__criterion', 'multi_clf__estimator__max_depth', 'multi_clf__estimator__max_features', 'multi_clf__estimator__max_leaf_nodes', 'multi_clf__estimator__min_impurity_decrease', 'multi_clf__estimator__min_impurity_split', 'multi_clf__estimator__min_samples_leaf', 'multi_clf__estimator__min_samples_split', 'multi_clf__estimator__min_weight_fraction_leaf', 'multi_clf__estimator__n_estimators', 'multi_clf__estimator__n_jobs', 'multi_clf__estimator

In [ ]:
parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................
[CV] ...................... , score=0.22017088800732376, total= 2.7min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] ...................... , score=0.22978333841928594, total= 2.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV] ...................... , score=0.22642660970399756, total= 2.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min finished


In [32]:
cv.best_params_

{}

In [33]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

             precision    recall  f1-score   support

  related-0       0.55      0.32      0.41      1565
  related-1       0.81      0.91      0.86      4941

avg / total       0.74      0.77      0.75      6506

             precision    recall  f1-score   support

  request-0       0.89      0.98      0.93      5420
  request-1       0.81      0.39      0.53      1134

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6524
    offer-1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

               precision    recall  f1-score   support

aid_related-0       0.71      0.89      0.79      3900
aid_related-1       0.75      0.46      0.57      2654

  avg / total       0.73      0.72      0.70      6554

                precision    recall  f1-score   support

medical_help-0       0.93      1.00      0.96      6038
medical_help-1     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [122]:
# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

['initial_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.